<a href="https://colab.research.google.com/github/c-daly/Notebooks/blob/main/EntropyCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a educational project to better understand entropy and decision trees as described in Chapter 14 of Norvig and Russel's AIMA.  

In [85]:
import numpy as np
import pandas as pd

label_column = 'willwait'

example1 = {'alt': 'yes', 'bar': 'no', 'frisat': 'no', 'hungry': 'yes', 'patrons': 'some', 'price': '3', 'rain': 'no', 'res': 'yes', 'type': 'french', 'wait': '10', 'willwait': 'yes'  }
example2 = {'alt': 'yes', 'bar': 'no', 'frisat': 'no', 'hungry': 'yes', 'patrons': 'full', 'price': '1', 'rain': 'no', 'res': 'no', 'type': 'thai', 'wait': '60', 'willwait': 'no'  }
example3 = {'alt': 'no', 'bar': 'yes', 'frisat': 'no', 'hungry': 'no', 'patrons': 'some', 'price': '1', 'rain': 'no', 'res': 'no', 'type': 'burger', 'wait': '10', 'willwait': 'yes'  }
example4 = {'alt': 'yes', 'bar': 'no', 'frisat': 'yes', 'hungry': 'yes', 'patrons': 'full', 'price': '1', 'rain': 'yes', 'res': 'no', 'type': 'thai', 'wait': '30', 'willwait': 'yes'  }
example5 = {'alt': 'yes', 'bar': 'no', 'frisat': 'yes', 'hungry': 'no', 'patrons': 'full', 'price': '3', 'rain': 'no', 'res': 'yes', 'type': 'french', 'wait': '61', 'willwait': 'no'  }
example6 = {'alt': 'no', 'bar': 'yes', 'frisat': 'no', 'hungry': 'yes', 'patrons': 'some', 'price': '2', 'rain': 'yes', 'res': 'yes', 'type': 'italian', 'wait': '10', 'willwait': 'yes'  }
example7 = {'alt': 'no', 'bar': 'yes', 'frisat': 'no', 'hungry': 'no', 'patrons': 'none', 'price': '1', 'rain': 'yes', 'res': 'no', 'type': 'burger', 'wait': '10', 'willwait': 'no'  }
example8 = {'alt': 'no', 'bar': 'no', 'frisat': 'no', 'hungry': 'yes', 'patrons': 'some', 'price': '2', 'rain': 'yes', 'res': 'yes', 'type': 'thai', 'wait': '10', 'willwait': 'yes'  }
example9 = {'alt': 'no', 'bar': 'yes', 'frisat': 'yes', 'hungry': 'no', 'patrons': 'full', 'price': '1', 'rain': 'yes', 'res': 'no', 'type': 'burger', 'wait': '61', 'willwait': 'no'  }
example10 = {'alt': 'yes', 'bar': 'yes', 'frisat': 'yes', 'hungry': 'yes', 'patrons': 'full', 'price': '3', 'rain': 'no', 'res': 'yes', 'type': 'italian', 'wait': '30', 'willwait': 'no' }
example11 = {'alt': 'no', 'bar': 'no', 'frisat': 'no', 'hungry': 'no', 'patrons': 'none', 'price': '1', 'rain': 'no', 'res': 'no', 'type': 'thai', 'wait': '10', 'willwait': 'no'  }
example12 = {'alt': 'yes', 'bar': 'yes', 'frisat': 'yes', 'hungry': 'yes', 'patrons': 'full', 'price': '1', 'rain': 'no', 'res': 'no', 'type': 'burger', 'wait': '60', 'willwait': 'yes' }

data = [
  example1, example2, example3, example4, example5, example6, example7, example8, example9, example10, example11, example12
] 

df = pd.DataFrame.from_records(data)


In [86]:
class Node:
  def __init__(self, parent, attribute):
    self.parent = parent
    self.children = {}
    self.attribute = attribute

In [90]:
def allExamplesHaveSameClassification(examples):
   return len(set(examples[label_column])) == 1


def getPluralityValue(examples):
  num_positives = len(getPositiveValues(examples))

  if num_positives > len(examples) - num_positives:
    return 'yes'
  else:
    return 'no'

def getPositiveValues(examples):
  positive_examples = df[df[label_column] == 'yes']
  #print(f"Positive examples: {positive_examples}")
  return positive_examples

def getNumPositiveExamplesByAttributeValue(examples, attribute, value):
  #print(f"{feature},{value}")
  positive_examples = examples[(examples[attribute] == value) & (examples[label_column] == 'yes')]
  #print(f"Positive examples: {positive_examples}")
  return len(positive_examples)

def B(prob):
  #print(f"prob: {np.log2(prob)}")
  if prob == 0:
    result = 0
  else:
    firstLogResult = 0
    secondLogResult = 0
    if prob > 0:
      firstLogResult = np.log2(prob)
    if prob < 1:
      secondLogResult = np.log2(1-prob)
    result = -(prob * firstLogResult + (1-prob) * secondLogResult)
  #print(f"B result: {result}")
  return result

def remainder(examples, attribute, values):
  remainder = 0
  total_count = len(examples)
  total_positive = len(examples[examples[label_column] == 'yes'])
  value_positive_count = 0
  value_negative_count = 0
  for value in values:
    #print(f"in value loop for value, attribute: {value},{attribute}")
    value_count = len(df[df[attribute] == value])
    value_positive_count = getNumPositiveExamplesByAttributeValue(examples, attribute, value)
    #print(f"value count/total: {value_count}/{total_count}")
    #print(f"{attribute}.{value}: value positive count/value count: {value_positive_count}/{value_count}")
    #print(f"prob arg: {value_positive_count/value_count}")
    temp = (value_count/total_count) * B(value_positive_count/value_count)
    #print(f"temp result: {temp}")
    remainder += temp

  #print(f"total value positive count: {value_positive_count}")
  #print(f"{attribute}: remainder: {remainder}")
  return remainder

def calculateEntropy(examples, feature, values):

  probabilities = []

  for value in values:
    #print(f"in value loop for value, feature: {value},{feature}")
    num_positives = getNumPositiveExamplesByFeatureValue(examples, feature, value)
    num_negatives = len(examples) - num_positives

  prob_positive = num_positives / len(examples)
  #print(f"prob_positive: {prob_positive}")
  entropy = -(prob_positive * np.log2(prob_positive))
  return entropy

def calculateGainForAttribute(examples, attribute, values):
  total_count = len(examples)
  total_positive = len(getPositiveValues(examples))

  b = B(total_positive/total_count)
  result = b - remainder(examples, attribute, values) 
  print(f"{attribute} gain: {result}")
  return result

def getAttributesFromExamples(examples):
  if len(examples) == 0:
    return None
  else:
    return [key for key in examples[0].keys() if key != label_column]

def getAttributeValuesFromExamples(examples, attribute):
  if len(examples) == 0:
    return None
  else:
    #return set([example[attribute] for example in examples])
    return examples[attribute].unique()

def getMostImportantAttribute(examples):
  gain = 0
  best_attribute = None
  attributes = examples.columns #getAttributesFromExamples(examples)
  for attribute in attributes:
    if attribute == label_column:
      continue
    #getAttributeValuesFromExamples(examples, attribute)
    temp_gain = calculateGainForAttribute(examples, attribute, getAttributeValuesFromExamples(examples, attribute))
    if temp_gain > gain:
      gain = temp_gain
      best_attribute = attribute
  print(f"Attribute/gain: {best_attribute}/{gain}")
  return best_attribute


In [91]:
def learnDecisionTree(features, examples, parent_examples):
  if not examples.values.any():
    return getPluralityValue(parent_examples)
  elif allExamplesHaveSameClassification(examples):
    print(examples[label_column])
    return examples[label_column].first()
  elif not features.any():
    return getPluralityValue(examples)
  else:
    attribute = getMostImportantAttribute(examples)
    if attribute is None:
      return
    node = Node(None, attribute)
    print(attribute)
    values = examples[attribute].unique()
    if values.any():
      for value in values:
        value_examples = examples.loc[examples[attribute] == value]
        #child = learnDecisionTree(features, value_examples, examples)
learnDecisionTree(df.columns, df, None)

alt gain: 0.0
bar gain: 0.0
frisat gain: 0.020720839623908027
hungry gain: 0.19570962879973086
patrons gain: 0.5408520829727552
price gain: 0.19570962879973086
rain gain: 0.020720839623908027
res gain: 0.020720839623908027
type gain: 1.1102230246251565e-16
wait gain: 0.20751874963942196
Attribute/gain: patrons/0.5408520829727552
patrons
